In [1]:
import openpyxl
import pandas as pd
import os

In [2]:
excelname='raw_data'
sheetname='kd'
wb=openpyxl.load_workbook(str(excelname)+'.xlsx')
ws=wb[str(sheetname)]

In [3]:
dataset=[]
i=1
while(ws.cell(row=i,column=1).value!=None):
    row=[]
    j=1
    while(ws.cell(row=i,column=j).value!=None):
        row.append(ws.cell(row=i,column=j).value)
        j+=1
    dataset.append(row)
    i+=1

In [9]:
def hyper(dataset,aroon_index):
    trade=[[0 for i in range(16)]for j in range(len(dataset))]
    trade[0][0]='date'
    trade[0][1]='open'
    trade[0][2]='high'
    trade[0][3]='low'
    trade[0][4]='close'
    trade[0][5]='Aroon_up'
    trade[0][6]='Aroon_down'
    trade[0][7]='Aroon_Oscillator'
    trade[0][8]='buy'
    trade[0][9]='hold'
    trade[0][10]='sell'
    trade[0][11]='number'
    trade[0][12]='value'
    trade[0][13]='period_return'
    trade[0][14]='holding_period'
    trade[0][15]='annual_return'
    for i in range(1,len(trade)):
        trade[i][0]=dataset[i][0]
        trade[i][1]=dataset[i][1]
        trade[i][2]=dataset[i][2]
        trade[i][3]=dataset[i][3]
        trade[i][4]=float(dataset[i][4])
    #Aroon Oscillator
    for i in range(aroon_index+1,len(trade)):
        high=trade[i][2]
        low=trade[i][3]
        for j in range(0,aroon_index+1):
            if(trade[i-j][2]>high):
                high_index=j
            if(trade[i-j][3]<low):
                low_index=j
        trade[i][5]=(aroon_index-high_index)/aroon_index*100
        trade[i][6]=(aroon_index-low_index)/aroon_index*100
        trade[i][7]=trade[i][5]-trade[i][6]
    # trading strategy:
    for i in range(aroon_index+1,len(trade)):
        if(trade[i][7]>0)and(trade[i-1][9]==0):
            trade[i][8]=1
            trade[i][9]=1
            trade[i][10]=0
        elif(trade[i][7]<0)and(trade[i-1][9]==1):
            trade[i][8]=0
            trade[i][9]=0
            trade[i][10]=1
        elif(trade[i-1][9]>0):
            trade[i][8]=0
            trade[i][9]=1
            trade[i][10]=0
    initial=1000
    for i in range(aroon_index+1,len(trade)):
        if(trade[i][8]==1)and(trade[i][9]==1)and(trade[i][10]==0):
            trade[i][11]=initial/trade[i][4]
            trade[i][12]=trade[i][11]*trade[i][4]
            trade[i][13]=round((trade[i][11]*trade[i][4]-initial)/initial,4)
        elif(trade[i][8]==0)and(trade[i][9]==0)and(trade[i][10]==1):
            trade[i][11]=0
            trade[i][12]=trade[i-1][11]*trade[i][4]
            trade[i][13]=round((trade[i-1][11]*trade[i][4]-initial)/initial,4)
        elif(trade[i][8]==0)and(trade[i][9]==1)and(trade[i][10]==0):
            trade[i][11]=trade[i-1][11]
            trade[i][12]=trade[i][11]*trade[i][4]
            trade[i][13]=round((trade[i][11]*trade[i][4]-initial)/initial,4)
            
    import datetime
    from datetime import date
    for i in range(1,len(trade)):
        trade[i][0]=datetime.datetime.strptime(trade[i][0],'%Y/%m/%d')
    
    first=0
    end=0
    for i in range(1,len(trade)):
        if(trade[i][8]==1):
            trade[i][14]=1
            first=trade[i][0]
        elif((trade[i][9]==1)or(trade[i][10]==1)and(trade[i][8]!=1)):
            end=trade[i][0]
            trade[i][14]=(end-first).days+1 
        
    for i in range(1,len(trade)):
        if(trade[i][14]!=0):
            trade[i][15]=(1+trade[i][13])**(365/trade[i][14])-1
            
    annual_return=1
    trading=0
    for i in range(1,len(trade)):
        if(trade[i][10]==1):
            annual_return*=(1+trade[i][13])
            trading+=1
    final_return=annual_return**(1/trading)-1
    return final_return
#     return trade

In [10]:
for i in range(10,31):
    print("The final_return of Aroon Oscillator "+str(i)+" is "+str(hyper(dataset,i)))

The final_return of Aroon Oscillator 10 is -0.001268867356664316
The final_return of Aroon Oscillator 11 is -0.0005790864747057256
The final_return of Aroon Oscillator 12 is -0.0008146424458332913
The final_return of Aroon Oscillator 13 is -0.0013274202433101179
The final_return of Aroon Oscillator 14 is 0.001129143580606895
The final_return of Aroon Oscillator 15 is 0.0031313264475729685
The final_return of Aroon Oscillator 16 is 0.00455928704710451
The final_return of Aroon Oscillator 17 is 0.004095903333253048
The final_return of Aroon Oscillator 18 is 0.004967995263354119
The final_return of Aroon Oscillator 19 is 0.0012513116076700026
The final_return of Aroon Oscillator 20 is 0.003901840546789659
The final_return of Aroon Oscillator 21 is 0.004109108240597292
The final_return of Aroon Oscillator 22 is 0.0009752019415780744
The final_return of Aroon Oscillator 23 is -0.000322437640235651
The final_return of Aroon Oscillator 24 is 0.0029947321704710816
The final_return of Aroon Osc

In [7]:
pd.DataFrame(hyper(dataset,25)).head(40)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,date,open,high,low,close,Aroon_up,Aroon_down,Aroon_Oscillator,buy,hold,sell,number,value,period_return,holding_period,annual_return
1,2003-06-30 00:00:00,37.1,37.4,36.92,37.08,0,0,0,0,0,0,0,0,0,0,0
2,2003-07-01 00:00:00,37.09,38.1,37.09,38.05,0,0,0,0,0,0,0,0,0,0,0
3,2003-07-02 00:00:00,38.17,38.82,38.1,38.69,0,0,0,0,0,0,0,0,0,0,0
4,2003-07-03 00:00:00,40.6,40.6,38.81,39,0,0,0,0,0,0,0,0,0,0,0
5,2003-07-04 00:00:00,39.1,39.26,38.75,39.26,0,0,0,0,0,0,0,0,0,0,0
6,2003-07-07 00:00:00,39.6,41,39.6,41,0,0,0,0,0,0,0,0,0,0,0
7,2003-07-08 00:00:00,42,42,40.7,41.19,0,0,0,0,0,0,0,0,0,0,0
8,2003-07-09 00:00:00,41.3,41.3,40.81,41.22,0,0,0,0,0,0,0,0,0,0,0
9,2003-07-10 00:00:00,41.2,41.2,40.05,40.05,0,0,0,0,0,0,0,0,0,0,0
